In [1]:
import numpy as np
import pandas as pd
import os
import json
import requests

In [2]:
toronto_bike_df = pd.read_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/toronto_bike_df.csv')
toronto_bike_df['ll'] = toronto_bike_df['lat'].astype(str) + ',' + toronto_bike_df['lon'].astype(str)
toronto_bike_df

,lat,lon,num_of_bikes,ll
0,43.665269,-79.319796,19,"43.665269,-79.319796"
1,43.671420,-79.445947,15,"43.67142,-79.445947"
2,43.666224,-79.317693,23,"43.666224,-79.317693"
3,43.653236,-79.376716,24,"43.653236,-79.376716"
4,43.663722,-79.380288,13,"43.663722,-79.380288"
...,...,...,...,...
648,43.772445,-79.511912,15,"43.772445,-79.511912"
649,43.653359,-79.365023,11,"43.653359,-79.365023"
650,43.706436,-79.251044,11,"43.70643614257429,-79.25104443353288"
651,43.655972,-79.370638,15,"43.655972,-79.370638"


In [36]:
def fsquare_stations(coordinates):
    """Put in coordinates to get general information about address within 1000m"""
    url = 'https://api.foursquare.com/v3/places/search'
    categories = '19046'
    foursquare_key = os.environ["FOUR_SQUARE_API"]
    params = {'ll': coordinates,
            'radius': 1000,
            'categories': categories,
            'limit': 50,
            'fields': 'name,location,rating'}
    headers = {'Accept': 'application/json', 'Authorization': foursquare_key}
    response = requests.get(url, params=params, headers=headers).json()
    return response

In [44]:
x = fsquare_stations('43.665269,-79.319796')

In [45]:
x

{'results': [],
 'context': {'geo_bounds': {'circle': {'center': {'latitude': 43.665269,
     'longitude': -79.319796},
    'radius': 1000}}}}

In [47]:
# Metro stations
def fsquare_stations_info(coordinates):
    """Put in coordinates to get a dictionary with information about the name, address and number of subways within 1000m"""
    url = 'https://api.foursquare.com/v3/places/search'
    categories = 19046 
    foursquare_key = os.environ["FOUR_SQUARE_API"]
    params = {'ll': coordinates,
            'radius': 1000,
            'categories': categories,
            'limit': 50,
            'fields': 'name,location'}
    headers = {'Accept': 'application/json', 'Authorization': foursquare_key}
    response = requests.get(url, params=params, headers=headers).json()
    subway_dict ={'name':[], 'address': [], 'num_of_stations': [], 'coordinates': []}
    if len(response['results']) == 0:
        subway_dict['name'].append(np.nan)
        subway_dict['address'].append(np.nan)
        subway_dict['num_of_stations'].append(0)
        subway_dict['coordinates'].append(coordinates)
    else:
        for subway in response['results']:
            if 'address' not in subway['location']:
                subway_dict['address'].append(np.nan)
            else:
                subway_dict['address'].append(subway['location']['address'])
            subway_dict['name'].append(subway['name'])
            subway_dict['num_of_stations'].append(len(response['results']))
            subway_dict['coordinates'].append(coordinates)
    return subway_dict

In [48]:
lst_toronto_stations_info = []
for location in toronto_bike_df['ll']:
    lst_toronto_stations_info.append(fsquare_stations_info(location))

In [49]:
lst_fsquare_df = []
for data in lst_toronto_stations_info:
    lst_fsquare_df.append(pd.DataFrame(data))

In [50]:
fsquare_df = pd.concat(lst_fsquare_df)
fsquare_df.to_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/fsquare_transit_df.csv', index=False)
fsquare_df

,name,address,num_of_stations,coordinates
0,NaN,NaN,0,"43.665269,-79.319796"
0,NaN,NaN,0,"43.67142,-79.445947"
0,NaN,NaN,0,"43.666224,-79.317693"
0,Dundas Subway Station,300 Yonge St,6,"43.653236,-79.376716"
1,King Subway Station,70 Yonge St,6,"43.653236,-79.376716"
...,...,...,...,...
0,Dundas Subway Station,300 Yonge St,2,"43.655972,-79.370638"
1,King Subway Station,70 Yonge St,2,"43.655972,-79.370638"
0,Wellesley Subway Station,551 Yonge St,3,"43.666874,-79.381155"
1,Bloor-Yonge Subway Station,20 Bloor St E,3,"43.666874,-79.381155"


In [40]:
def yelp_stations(coordinates):
    """Put in coordinates to get information about the name,address,ratings,stats, price and popularity within 1000m"""
    url = 'https://api.yelp.com/v3/businesses/search'
    categories = 'trainstations, All' #dining and drinking restaurant
    yelp_key = os.environ['yelp_key']
    params = {'latitude': coordinates.split(',')[0],
            'longitude': coordinates.split(',')[1],
            'radius': 1000,
            'limit': 50,
            'categories': categories}
    headers = {'Accept': 'application/json', 'Authorization': "Bearer " + yelp_key}
    response = requests.get(url, params=params, headers=headers).json()
    return response

In [43]:
x = yelp_stations('43.665269,-79.319796')
x

{'businesses': [],
 'total': 0,
 'region': {'center': {'longitude': -79.319796, 'latitude': 43.665269}}}

In [51]:
def yelp_station_details(coordinates):
    """Put in coordinates to get information about the name,address,ratings,stats, price and popularity within 1000m"""
    url = 'https://api.yelp.com/v3/businesses/search'
    categories = 'trainstations,All' #dining and drinking restaurant
    yelp_key = os.environ['yelp_key']
    params = {'latitude': coordinates.split(',')[0],
            'longitude': coordinates.split(',')[1],
            'radius': 1000,
            'limit': 50,
            'categories': categories}
    headers = {'Accept': 'application/json', 'Authorization': "Bearer " + yelp_key}
    response = requests.get(url, params=params, headers=headers).json()
    yelp_station_details = {'name': [], 'address': [], 'yelp_num_stations': [], 'coordinates': []}
    for locations in response['businesses']:
        if len(response['businesses']) == 0:
            yelp_station_details['name'].append(np.nan)
            yelp_station_details['address'].append(np.nan)
            yelp_station_details['yelp_num_stations'].append(0)
            yelp_station_details['coordinates'].append(coordinates)
        else:
            yelp_station_details['name'].append(locations['name'])
            yelp_station_details['address'].append(locations['location']['display_address'][0])
            yelp_station_details['yelp_num_stations'].append(len(response['businesses']))
            yelp_station_details['coordinates'].append(coordinates)
    return yelp_station_details

In [52]:
list_yelp_details = []
for location in toronto_bike_df['ll']:
    print(location)
    list_yelp_details.append(yelp_station_details(location))

43.665269,-79.319796
43.67142,-79.445947
43.666224,-79.317693
43.653236,-79.376716
43.663722,-79.380288
43.657763,-79.389165
43.656729,-79.382736
43.664467,-79.414783
43.675492,-79.388858
43.674991,-79.396273
43.671944,-79.387778
43.662365,-79.351104
43.646144,-79.377962
43.663993,-79.358534
43.657266,-79.374756
43.6672144,-79.3452697
43.673939,-79.441159
43.657991,-79.340075
43.66745,-79.329408
43.672453,-79.338259
43.6592088,-79.3348396
43.689211,-79.35438
43.655431,-79.380653
43.656927,-79.378497
43.661324,-79.449534
43.658295,-79.372346
43.6940155,-79.2889842
43.6809164,-79.4229684
43.6910667,-79.39535
43.6975969,-79.39654
43.639832,-79.395954
43.667158,-79.402761
43.667333,-79.399429
43.6480008,-79.383177
43.656518,-79.389099
43.658148,-79.398167
43.687999,-79.38906
43.63796,-79.387502
43.6563889,-79.45166666666667
43.639179,-79.399595
43.6322445,-79.4108105
43.6616667,-79.40138888888889
43.66666667,-79.45833333333334
43.657192,-79.452559
43.6576,-79.4032
43.660694,-79.379052
43.6

In [53]:
yelp_df_list = []
for location in list_yelp_details:
    yelp_df_list.append(pd.DataFrame(location))

In [54]:
yelp_df = pd.concat(yelp_df_list)
yelp_df.to_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/yelp_transit_df.csv', index=False)
yelp_df

,name,address,yelp_num_stations,coordinates
0,Union Station,65 Front Street W,7.0,"43.653236,-79.376716"
1,VIA Rail Canada,65 Front Street W,7.0,"43.653236,-79.376716"
2,Queen Station,171 Yonge Street,7.0,"43.653236,-79.376716"
3,Dundas Station,"Toronto, ON M5B 1J3",7.0,"43.653236,-79.376716"
4,St Patrick Subway Station,449 University Avenue,7.0,"43.653236,-79.376716"
...,...,...,...,...
1,Bloor-Yonge Subway Station,20 Bloor Street E,6.0,"43.666874,-79.381155"
2,Wellesley Subway Station,16 Wellesley Street E,6.0,"43.666874,-79.381155"
3,Dundas Station,"Toronto, ON M5B 1J3",6.0,"43.666874,-79.381155"
4,Bay Subway Station,1240 Bay Street,6.0,"43.666874,-79.381155"


In [56]:
yelp_df.describe()

,yelp_num_stations
count,1248.000000
mean,4.480769
std,2.050591
min,1.000000
25%,3.000000
50%,5.000000
75%,6.000000
max,8.000000


In [57]:
fsquare_df.describe()

,num_of_stations
count,1260.000000
mean,2.744444
std,2.142225
min,0.000000
25%,0.000000
50%,3.000000
75%,4.000000
max,7.000000
